#### 전략 1 [3일 모멘텀 단기 전략]  
마켓타이밍 ==True  
거래대금 100억이상  
3일째 가격상승이 되는 종목 매수후 3일뒤 매도  

3일간 상승한 모멘텀이 앞으로 3일간 지속될 것 이라는 가정  

In [6]:
import pandas as pd 
df_raw = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/data Row/mom.csv', index_col=0)

In [7]:
df_raw = df_raw.drop(df_raw.iloc[:,8:],axis=1)

In [12]:
# make momentum_signal
def make_momentum_signal(df):
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        con1 = (df.loc[idx[0]:idx[-1],'종가'].shift(-3))
        con2 = (df.loc[idx[0]:idx[-1],'종가'].shift(-2))
        con3 = (df.loc[idx[0]:idx[-1],'종가'].shift(-1))
        df.loc[idx[0]:idx[-1], 'momentum'] = (con1<con2)&(con2<con3)&(con3<(df.loc[idx[0]:idx[-1], '종가']))
    return df
        

In [10]:
df = make_momentum_signal(df_raw)

In [13]:
# market_timming 병합작업
def making_mt(start,end,code):
    from pykrx import stock
    df = stock.get_index_fundamental(start,end,code)
    df = df.drop(['등락률','PER','선행PER','PBR','배당수익률'],axis=1)
    
    ''' code : KOSPI ('1001') KOSDAQ ('2001') '''
    if code=='1001': 
        df['sma20'] = df['종가'].rolling(20).mean()
        df['mt'] = df['종가']>df['sma20']
        df = df.dropna()
        return df
    elif code=='2001':     
        df['sma3'] = df['종가'].rolling(3).mean()
        df['sma5'] = df['종가'].rolling(5).mean()
        df['sma10'] = df['종가'].rolling(10).mean()
        df['mt'] = ((df['종가']>df['sma3'])|(df['종가']>df['sma5'])|(df['종가']>df['sma10']))
        df = df.dropna()
        return df
    else:
        print('올바른 종목 code 입력')

In [23]:
start = df[df['종목명']=='삼성전자'].head(1)['날짜'].values[0]
end = df[df['종목명']=='삼성전자'].tail(1)['날짜'].values[0]

In [25]:
df_mt.reset_index(drop=True, inplace=True)

In [87]:
df_mt

,종가,sma3,sma5,sma10,mt,날짜
0,842.60,844.833333,850.178,882.014,False,2018-02-09 00:00:00
1,843.24,849.260000,847.182,873.633,False,2018-02-12 00:00:00
2,829.39,838.410000,841.426,864.476,False,2018-02-13 00:00:00
3,848.03,840.220000,845.040,857.922,True,2018-02-14 00:00:00
4,875.81,851.076667,847.814,854.683,True,2018-02-19 00:00:00
...,...,...,...,...,...,...
1199,691.25,703.990000,706.464,713.179,False,2022-12-23 00:00:00
1200,694.68,700.316667,701.956,711.125,False,2022-12-26 00:00:00
1201,704.19,696.706667,702.168,710.028,True,2022-12-27 00:00:00
1202,692.37,697.080000,699.502,706.365,False,2022-12-28 00:00:00


In [24]:
df_mt = making_mt(start,end,'2001')
df_mt['날짜']=df_mt.index

In [34]:
df_mt['날짜'] = df_mt['날짜'].astype('object')

In [41]:
df_join = pd.merge(df,df_mt,how='left', on='날짜')

#### 거래대금 필터만우선적용..

In [49]:
df['vol_condition'] = df['누적거래대금'] >=10000000000

#### 최종시그널

In [52]:
def make_buysell(df):
    df['buy'] = (df['vol_condition'])&(df['momentum'])
    object_list = df['종목명'].unique()
    for i in object_list:
        idx = df[df['종목명']==i].index
        df.loc[idx[0]:idx[-1],'sell'] = df.loc[idx[0]:idx[-1],'buy'].shift(3)
    return df

In [53]:
df = make_buysell(df)

In [55]:
code_kospi = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kospi_codes.csv')
code_kosdaq = pd.read_csv('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test/SRC/current_kosdaq_codes.csv')

In [56]:
code = pd.concat([code_kospi,code_kosdaq])
code.rename(columns={'회사명':'종목명'},inplace=True)
code['종목코드'] = code['종목코드'].astype('str')

In [58]:
df = df.merge(code,on='종목명',how='inner')
df.rename(columns={'종목코드':'코드'},inplace=True)
df['구매수량']=3

In [71]:
df['구매수량']=1

#### 백테스트

In [60]:
df['sell'].unique()

array([nan, True, False], dtype=object)

In [61]:
import sys
sys.path.append('C:/Users/mi-bi/OneDrive/바탕 화면/Pjt/Kis_Api_test')
from BackTesting_System.backtesting import BackTesting, Utils

In [72]:
bt1 = BackTesting()
ut = Utils()

In [73]:
bt1.MONEY = 10000000

In [74]:
bt1.buy_sell(df=df,buy='buy',sell='sell')

In [75]:
bt1.update_information()

In [76]:
bt1.JOUNAL

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2018-02-01,285130,74726,1,105.0,매수
1,2018-02-01,251270,173500,1,244.0,매수
2,2018-02-01,267250,92600,1,130.0,매수
3,2018-02-01,267260,55634,1,78.0,매수
4,2018-02-01,207940,438309,1,616.0,매수
...,...,...,...,...,...,...
216,2018-07-24,023440,2235,1,3.0,매수
217,2019-05-13,900250,2080,1,3.0,매수
218,2019-05-14,900250,2010,1,3.0,매도
219,2019-08-07,033170,1025,1,1.0,매수


In [77]:
bt1.ECONO_INFORM

,날짜,매수총합,매도총합,매매비용,실현손익
0,2018-02-01,9431159,0.0,13255.0,0.0
1,2018-02-02,0.0,270324,379.0,-9774
2,2018-02-05,594836,226000,1154.0,-22500
3,2018-02-06,1446092,1371900,3962.0,-142083
4,2018-02-08,317091,102571,590.0,-5225
5,2018-02-09,56700,47404,146.0,-12479
6,2018-02-12,30079,0.0,43.0,0.0
7,2018-02-13,117631,0.0,164.0,0.0
8,2018-02-19,0.0,33800,47.0,-2375.0
9,2018-02-20,12639,3920,24.0,-410


In [78]:
bt1.ECONO_INFORM['실현손익'].sum()

-195366.0

In [83]:
bt1.ECONO_INFORM['실현손익'].sum()/10000000*100

-1.9536600000000002

#### 문제 : 매수종목이 너무많음 5일만에 투자가 끝남  
해결책 : 매수종목을 더 추려야함 + 기술적지표 추가 + 종목별 비중 + 마켓타이밍

In [86]:
df[df['날짜']=='2018-02-01']

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,momentum,vol_condition,buy,sell,코드,상장일,구매수량
33,2018-02-01,55800,53700,56300,53700,153868,8553028100,LX세미콘,False,False,False,False,108320,2022-11-03,1
6379,2018-02-01,42050,44050,44100,41950,681048,28919158000,PI첨단소재,False,True,False,False,178920,2021-08-09,1
8297,2018-02-01,16733,16900,16966,16300,144382,2405716400,엠씨넥스,False,False,False,False,97520,2021-07-06,1
26208,2018-02-01,70082,67229,70367,66563,353899,24408394250,콘텐트리중앙,False,True,False,False,36420,2019-10-18,1
27421,2018-02-01,46930,46226,47400,46226,367097,17210954800,포스코케미칼,False,True,False,False,3670,2019-05-29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538833,2018-02-01,26850,26100,26850,26000,16547,438449750,하이록코리아,False,False,False,False,13030,1989-12-15,1
2540046,2018-02-01,1915,1495,1915,1480,81618848,140639087615,SBI인베스트먼트,False,True,False,False,19550,1989-09-06,1
2541259,2018-02-01,802,716,890,703,22363176,18051860693,리더스 기술투자,True,True,True,False,19570,1989-03-14,1
2542472,2018-02-01,1637,1383,1715,1366,5602004,8802460838,엠벤처투자,True,False,False,False,19590,1989-03-14,1


#### 매도조건 바꾸기 : 현재 3일뒤 매도, 